<a href="https://mumbrella.com.au/wp-content/uploads/2013/01/Screen-Shot-2013-01-22-at-4.25.45-PM.png"><center><img src="https://assets.2ser.com/wp-content/aws/uploads/2018/02/02181138/Melbourne-VS-Sydney-990x500.png" width = 700 h></center> </a>

<h1 align=center><font size = 10>Settling the Great Australian Rivalry </font></h1>

## Introduction

In this analysis we will aim to quantify the relative strengths of both Melbourne and Sydney in the hope that we can (finally) state definitively whether Sydney is better than Melbourne.

## Table of Contents

[1. Data](#Data)  
[2. Clustering](#Clustering)

## Initialisation

Before starting the analysis, load all the required libraries.

In [1]:
# load standard libraries
import pandas as pd
import numpy as np

# load libraries for reading html requests
from urllib.request import Request, urlopen
import requests

# install mapping library
import folium

# install and load library to calulate distances
#!pip install haversine
import haversine

## Data

Download the data and prepare it for the analysis.

Read in the csv of Australian postodes from Matthew Proctor (https://www.matthewproctor.com/)

In [2]:
# provide the file link
link_aus_postcodes = 'https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv'

# set a new header so website allows reading of file
df_aus_postcodes = Request(link_aus_postcodes)
df_aus_postcodes.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
df_aus_postcodes = urlopen(df_aus_postcodes)

# download file
df_aus_postcodes = pd.read_csv(df_aus_postcodes)

# show sample of results
print(df_aus_postcodes.shape)
df_aus_postcodes.head()

(18275, 14)


,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region
0,230,200,ANU,ACT,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R1
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,NaN,NaN,NaN,R1
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,Darwin City,701.0,Darwin,R1
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,Darwin City,701.0,Darwin,R1
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,Darwin Suburbs,701.0,Darwin,R1


Only a subset of NSW and VIC postcodes are required.

In [3]:
# subset to NSW and VIC
df_postcodes = df_aus_postcodes[(df_aus_postcodes['state'].isin(['NSW', 'VIC']))]

# ensure the postcodes are delivery areas and in region R1 (city)
df_postcodes  = df_postcodes[(df_postcodes['region']=='R1') & 
                             (df_postcodes['type']=='Delivery Area')]

# subset to the most central postcodes for the city (ie exclude outlying suburbs)
central_postcodes_sydney = ['Sydney - City and Inner South', 
                            'Sydney - Eastern Suburbs',
                            'Sydney - Inner South West', 
                            'Sydney - Inner West']
central_postcodes_melbourne = ['Melbourne - Inner',
                               'Melbourne - Inner East', 
                               'Melbourne - Inner South', 
                               'Melbourne - West']

df_postcodes = df_postcodes[(df_postcodes['sa4name'].isin(central_postcodes_sydney)) | 
                            (df_postcodes['sa4name'].isin(central_postcodes_melbourne))]

# show sample of results
print(df_postcodes.shape)
df_postcodes.head()

(472, 14)


,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region
689,20208,2000,BARANGAROO,NSW,151.201580,-33.860520,Sydney Metro,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
691,4479,2000,DAWES POINT,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
692,4480,2000,HAYMARKET,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
693,4481,2000,MILLERS POINT,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
694,4482,2000,PARLIAMENT HOUSE,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1


Keep only required data and collapse results to postcode.

In [4]:
# retain only required data
df_postcodes = df_postcodes[['postcode', 'locality', 'state', 'long', 'lat', 'sa4name']]

# some postcodes have different lat, longs so take average to standardise
df_pc_latlongs = df_postcodes.groupby('postcode')['lat', 'long'].mean().reset_index()
df_postcodes = df_postcodes[['postcode', 'locality', 'state', 'sa4name']].merge(df_pc_latlongs, how='left', on='postcode')

# create suburbs names - potentially multiple suburbs
df_postcodes['suburb'] = df_postcodes.groupby(['postcode','state','long', 'lat', 'sa4name'])['locality'].transform(lambda x: ', '.join(x))

# rename SA4 name
df_postcodes['lga'] = df_postcodes['sa4name']

# remove duplicates for postcodes
df_postcodes = df_postcodes[['postcode','suburb','lga','state','long', 'lat']].drop_duplicates().reset_index(drop=True)

# show sample of results
print(df_postcodes.shape)
df_postcodes.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


(182, 6)


,postcode,suburb,lga,state,long,lat
0,2000,"BARANGAROO, DAWES POINT, HAYMARKET, MILLERS PO...",Sydney - City and Inner South,NSW,151.249765,-33.860024
1,2006,THE UNIVERSITY OF SYDNEY,Sydney - City and Inner South,NSW,151.186507,-33.889219
2,2007,"BROADWAY, ULTIMO",Sydney - City and Inner South,NSW,151.196650,-33.883189
3,2008,"CHIPPENDALE, DARLINGTON",Sydney - City and Inner South,NSW,151.193858,-33.891146
4,2009,PYRMONT,Sydney - City and Inner South,NSW,151.193055,-33.871222


Extract the businesses and attractions in each of the postcodes, but first understand the spread of postcodes in each Local Government Area to set reasonable radii for the searches, as the geographic size of each city is different.

In [5]:
# estimate the average size of each postcode by LGA
df_sizes = df_postcodes.groupby(['state', 'lga']).agg({'state': 'count', 'lat': ['min','max'], 'long': ['min','max']}).reset_index()
df_sizes['tl'] = list(zip(df_sizes[('lat', 'max')], df_sizes[('long', 'min')])) 
df_sizes['tr'] = list(zip(df_sizes[('lat', 'max')], df_sizes[('long', 'max')])) 
df_sizes['bl'] = list(zip(df_sizes[('lat', 'min')], df_sizes[('long', 'min')])) 
df_sizes['width'] = df_sizes.apply(lambda row: haversine.haversine(row['tl'][0],row['tr'][0]), axis=1)  # km
df_sizes['height'] = df_sizes.apply(lambda row: haversine.haversine(row['tl'][0],row['bl'][0]), axis=1) # km
df_sizes['avg_area'] = df_sizes['width']*df_sizes['height']/df_sizes[('state', 'count')]                # km^2

# estimate the average radius of each postcode by lGA
df_radius = df_sizes.copy()
df_radius['avg_radius'] = 1000*(df_sizes['avg_area']/np.pi)**(1/2)                                       # m
df_radius = df_radius[['state', 'lga', 'avg_radius']]
df_radius.columns = ['state', 'count', 'lga', 'radius']
df_radius = df_radius[['lga', 'radius']]

# display calculated radii
df_radius

,lga,radius
0,Sydney - City and Inner South,1116.630733
1,Sydney - Eastern Suburbs,1143.492966
2,Sydney - Inner South West,1479.142526
3,Sydney - Inner West,1202.087786
4,Melbourne - Inner,1087.693641
5,Melbourne - Inner East,1585.391794
6,Melbourne - Inner South,1955.472186
7,Melbourne - West,3267.340677


Display the postcodes for each of the cities independently, with radius.

In [6]:
# add radius to postcode data and calculate centres for Sydney and Melbourne
df_postcodes = df_postcodes.merge(df_radius, how='left')
syd_lat_lon = [df_postcodes[df_postcodes['state']=='NSW']['lat'].mean(), 
               df_postcodes[df_postcodes['state']=='NSW']['long'].mean()]
mel_lat_lon = [df_postcodes[df_postcodes['state']=='VIC']['lat'].mean(), 
               df_postcodes[df_postcodes['state']=='VIC']['long'].mean()]
print()
print(f'Sydney centre: {syd_lat_lon}')
print(f'Melbourne centre: {mel_lat_lon}')
print()
df_postcodes.head()


Sydney centre: [-33.909083297902875, 151.1511261108014]
Melbourne centre: [-37.83067081710809, 144.95407667383336]



,postcode,suburb,lga,state,long,lat,radius
0,2000,"BARANGAROO, DAWES POINT, HAYMARKET, MILLERS PO...",Sydney - City and Inner South,NSW,151.249765,-33.860024,1116.630733
1,2006,THE UNIVERSITY OF SYDNEY,Sydney - City and Inner South,NSW,151.186507,-33.889219,1116.630733
2,2007,"BROADWAY, ULTIMO",Sydney - City and Inner South,NSW,151.196650,-33.883189,1116.630733
3,2008,"CHIPPENDALE, DARLINGTON",Sydney - City and Inner South,NSW,151.193858,-33.891146,1116.630733
4,2009,PYRMONT,Sydney - City and Inner South,NSW,151.193055,-33.871222,1116.630733


In [7]:
# create map of Sydney using latitude and longitude values
map_sydney = folium.Map(location=syd_lat_lon, zoom_start=12)

# get Sydney data
df_syd = df_postcodes[df_postcodes['state']=='NSW']

# add colours to each lga
lga_syd = set(list(df_syd['lga']))
colours = ['red', 'green', 'blue', 'yellow']
lga_syd = dict(zip(lga_syd, colours))

# add markers to map
for lat, lng, lga, pc, suburb, radius in zip(df_syd['lat'], df_syd['long'], df_syd['lga'], df_syd['postcode'], df_syd['suburb'], df_syd['radius']):
    label = f'{pc} ({lga}): {suburb}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=lga_syd[lga],
        fill=True,
        fill_color=lga_syd[lga],
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    folium.Circle([lat, lng],
                    radius=radius,
                  color=lga_syd[lga]
                   ).add_to(map_sydney)

# display map
map_sydney

In [8]:
# create map of Melbourne using latitude and longitude values
map_melbourne = folium.Map(location=mel_lat_lon, zoom_start=12)

# get Sydney data
df_mel = df_postcodes[df_postcodes['state']=='VIC']

# add colours to each lga
lga_mel = set(list(df_mel['lga']))
colours = ['red', 'green', 'blue', 'yellow']
lga_mel = dict(zip(lga_mel, colours))

# add markers to map
for lat, lng, lga, pc, suburb, radius in zip(df_mel['lat'], df_mel['long'], df_mel['lga'], df_mel['postcode'], df_mel['suburb'], df_mel['radius']):
    label = f'{pc} ({lga}): {suburb}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=lga_mel[lga],
        fill=True,
        fill_color=lga_mel[lga],
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    folium.Circle([lat, lng],
                    radius=radius,
                  color=lga_mel[lga]
                   ).add_to(map_melbourne)

# display map
map_melbourne

Extract the businesses and attractions in each of the postcodes, taking into account the different radii for each LGA.

In [99]:
# read in credentials
cred = pd.read_csv('./FourSquare_Credentials.txt', header=0)
CLIENT_ID = cred.iloc[0, 0]
CLIENT_SECRET = cred.iloc[1, 0]
VERSION = cred.iloc[2, 0]
LIMIT = 100

In [100]:
# define function (from week three labs - modified to accept a dynamic radius)
def getNearbyVenues(names, latitudes, longitudes, radiuses):
    
    venues_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radiuses):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(url)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
# extract details for both cities
df_venues = getNearbyVenues(df_postcodes['postcode'], df_postcodes['lat'], df_postcodes['long'], df_postcodes['radius'])

In [104]:
# save data to csv so no need to run again
df_venues.to_csv('Venues_SydMel.csv')

In [107]:
# look at the number of venue categories
print(f"{len(df_venues['Venue Category'])} venues returned in {len(set(df_venues['Venue Category']))} categories")

11248 venues returned in 403 categories


In [108]:
# load both the venues and mapping from csv and combine
df_groups = pd.read_csv('Venues_SydMel.csv')
df_groups = df_groups.merge(pd.read_csv('VenuesGroups_SydMel.csv'), how='left')
df_groups.head()

,Unnamed: 0,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,0,2000,-33.860024,151.249765,Bradley's Head,-33.853210,151.246259,Scenic Lookout,Nature
1,1,2000,-33.860024,151.249765,Shark Island,-33.858324,151.257837,Harbor / Marina,Transport
2,2,2000,-33.860024,151.249765,Royal Motor Yacht Club,-33.867105,151.254417,Harbor / Marina,Transport
3,3,2000,-33.860024,151.249765,Duff Reserve,-33.864291,151.251022,Park,Nature
4,4,2000,-33.860024,151.249765,Royal Prince Edward Yacht Club,-33.865280,151.253120,Harbor / Marina,Transport
